# EDA : Flight Price Prediction Dataset

In [312]:
train = pd.read_excel('C:\\Users\\Snehal Sanap\\OneDrive\\Desktop\\AIML_STUDY\\INeuron\\Community Live Sessions\\Flight_Price\\Data_Train.xlsx')
test = pd.read_excel('C:\\Users\\Snehal Sanap\\OneDrive\\Desktop\\AIML_STUDY\\INeuron\\Community Live Sessions\\Flight_Price\\Test_set.xlsx')
train.shape, test.shape

((10683, 11), (2671, 10))

In [313]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


In [314]:
train[~train.apply(np.isreal).all(1)]

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price


In [315]:
train.isnull().sum()

Airline            0
Date_of_Journey    0
Source             0
Destination        0
Route              1
Dep_Time           0
Arrival_Time       0
Duration           0
Total_Stops        1
Additional_Info    0
Price              0
dtype: int64

In [316]:
train.describe().transpose()
#price target col is right skewed

,count,mean,std,min,25%,50%,75%,max
Price,10683.0,9087.064121,4611.359167,1759.0,5277.0,8372.0,12373.0,79512.0


In [317]:
# priniting the details about the datasets in a summary format

print("\nEDA on Training Set\n")
print("#"*30)

print("\nFeatures/Columns : \n", train.columns)
print("\n\nNumber of Features/Columns : ", len(train.columns))
print("\nNumber of Rows : ",len(train))
print("\n\nData Types :\n", train.dtypes)

print("\n Contains NaN/Empty cells : ", train.isnull().values.any())

print("\n Total empty cells by column :\n", train.isnull().sum(), "\n\n")


# Test Set
print("#"*30)
print("\nEDA on Test Set\n")
print("#"*30)


print("\nFeatures/Columns : \n",test.columns)
print("\n\nNumber of Features/Columns : ",len(test.columns))
print("\nNumber of Rows : ",len(test))
print("\n\nData Types :\n", test.dtypes)
print("\n Contains NaN/Empty cells : ", test.isnull().values.any())
print("\n Total empty cells by column :\n", test.isnull().sum())


EDA on Training Set

##############################

Features/Columns : 
 Index(['Airline', 'Date_of_Journey', 'Source', 'Destination', 'Route',
       'Dep_Time', 'Arrival_Time', 'Duration', 'Total_Stops',
       'Additional_Info', 'Price'],
      dtype='object')


Number of Features/Columns :  11

Number of Rows :  10683


Data Types :
 Airline            object
Date_of_Journey    object
Source             object
Destination        object
Route              object
Dep_Time           object
Arrival_Time       object
Duration           object
Total_Stops        object
Additional_Info    object
Price               int64
dtype: object

 Contains NaN/Empty cells :  True

 Total empty cells by column :
 Airline            0
Date_of_Journey    0
Source             0
Destination        0
Route              1
Dep_Time           0
Arrival_Time       0
Duration           0
Total_Stops        1
Additional_Info    0
Price              0
dtype: int64 


##############################

EDA on Test

In [318]:
nan_cols = [i for i in train.columns if train[i].isnull().any()]
nan_cols

['Route', 'Total_Stops']

In [319]:
num_cols = [i for i in train.select_dtypes(include='number').columns]
num_cols

['Price']

In [320]:
cat_cols = [i for i in train.select_dtypes(include='object').columns]
print(cat_cols, len(cat_cols))

['Airline', 'Date_of_Journey', 'Source', 'Destination', 'Route', 'Dep_Time', 'Arrival_Time', 'Duration', 'Total_Stops', 'Additional_Info'] 10


# FEATURE ENGINEERING

In [321]:
# Missing Values
for i in nan_cols:
    print(i, '    --> NA values :    {} , ({}%)'.format(train[i].isnull().sum(), np.round(train[i].isnull().mean(),4)))

Route     --> NA values :    1 , (0.0001%)
Total_Stops     --> NA values :    1 , (0.0001%)


#sinec the % of nan values is very less we can easily drop this 2 nan vavlues from the df

In [322]:
train.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [323]:
#Temporal Variable
# 1. Date of journey : convert fron obj to datetime dtype and the n extract necessary info from it
train['day_journey'] =pd.to_datetime(train['Date_of_Journey'], format='%d/%m/%Y').dt.day
train['month_journey'] =pd.to_datetime(train['Date_of_Journey'], format='%d/%m/%Y').dt.month
#this can also be by using string method -> .split('/') ---here it needs no datetime conversion also since after this split will convert to int
#datetime conversion needed if some forecatsing is purpose
#train[day] = train['Date_of_Journey'].str.split('/').str[0] #2nd str to iterate throuth the list of str values got by first split ...llly str[1], [2] month and year
#df["Date"]=df['Date_of_Journey'].apply(lambda x:x.split("/")[0])

In [324]:
pd.to_datetime(train['Date_of_Journey']).dt.year.unique()

array([2019], dtype=int64)

In [325]:
train.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,day_journey,month_journey
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897,24,3
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662,1,5
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882,9,6
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218,12,5
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302,1,3


In [326]:
train.drop(['Date_of_Journey'], axis=1, inplace = True)
train.head(3)

,Airline,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,day_journey,month_journey
0,IndiGo,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897,24,3
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662,1,5
2,Jet Airways,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882,9,6


In [327]:
#now apply same on test data
test['day_journey'] =pd.to_datetime(test['Date_of_Journey'], format='%d/%m/%Y').dt.day
test['month_journey'] =pd.to_datetime(test['Date_of_Journey'], format='%d/%m/%Y').dt.month
test.drop(['Date_of_Journey'], axis=1, inplace = True)
test.head(3)

,Airline,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,day_journey,month_journey
0,Jet Airways,Delhi,Cochin,DEL → BOM → COK,17:30,04:25 07 Jun,10h 55m,1 stop,No info,6,6
1,IndiGo,Kolkata,Banglore,CCU → MAA → BLR,06:20,10:20,4h,1 stop,No info,12,5
2,Jet Airways,Delhi,Cochin,DEL → BOM → COK,19:15,19:00 22 May,23h 45m,1 stop,In-flight meal not included,21,5


In [328]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2671 entries, 0 to 2670
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          2671 non-null   object
 1   Source           2671 non-null   object
 2   Destination      2671 non-null   object
 3   Route            2671 non-null   object
 4   Dep_Time         2671 non-null   object
 5   Arrival_Time     2671 non-null   object
 6   Duration         2671 non-null   object
 7   Total_Stops      2671 non-null   object
 8   Additional_Info  2671 non-null   object
 9   day_journey      2671 non-null   int64 
 10  month_journey    2671 non-null   int64 
dtypes: int64(2), object(9)
memory usage: 229.7+ KB


In [329]:
# 2. Dep_Time
train.Dep_Time.unique()

array(['22:20', '05:50', '09:25', '18:05', '16:50', '09:00', '18:55',
       '08:00', '08:55', '11:25', '09:45', '20:20', '11:40', '21:10',
       '17:15', '16:40', '08:45', '14:00', '20:15', '16:00', '14:10',
       '22:00', '04:00', '21:25', '21:50', '07:00', '07:05', '09:50',
       '14:35', '10:35', '15:05', '14:15', '06:45', '20:55', '11:10',
       '05:45', '19:00', '23:05', '11:00', '09:35', '21:15', '23:55',
       '19:45', '08:50', '15:40', '06:05', '15:00', '13:55', '05:55',
       '13:20', '05:05', '06:25', '17:30', '08:20', '19:55', '06:30',
       '14:05', '02:00', '09:40', '08:25', '20:25', '13:15', '02:15',
       '16:55', '20:45', '05:15', '19:50', '20:00', '06:10', '19:30',
       '04:45', '12:55', '18:15', '17:20', '15:25', '23:00', '12:00',
       '14:45', '11:50', '11:30', '14:40', '19:10', '06:00', '23:30',
       '07:35', '13:05', '12:30', '15:10', '12:50', '18:25', '16:30',
       '00:40', '06:50', '13:00', '19:15', '01:30', '17:00', '10:00',
       '19:35', '15:

In [330]:
train['hour'] = train['Dep_Time'].apply(lambda x:x.split(':')[0]) #train['Dep_Time'].apply(lambda x:int(x.split(':')[0]))
train['min'] = train['Dep_Time'].apply(lambda x:x.split(':')[1])
train.head()

,Airline,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,day_journey,month_journey,hour,min
0,IndiGo,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897,24,3,22,20
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662,1,5,05,50
2,Jet Airways,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882,9,6,09,25
3,IndiGo,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218,12,5,18,05
4,IndiGo,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302,1,3,16,50


In [331]:
train.drop('Dep_Time', axis=1, inplace = True)
#pd.to_datetime(train['Dep_Time']).dt.hour and minute

In [332]:
#same apply on test data
test['hour'] = test['Dep_Time'].apply(lambda x:x.split(':')[0])
test['min'] = test['Dep_Time'].apply(lambda x:x.split(':')[1])
test.drop('Dep_Time', axis=1, inplace = True)
test.head()

,Airline,Source,Destination,Route,Arrival_Time,Duration,Total_Stops,Additional_Info,day_journey,month_journey,hour,min
0,Jet Airways,Delhi,Cochin,DEL → BOM → COK,04:25 07 Jun,10h 55m,1 stop,No info,6,6,17,30
1,IndiGo,Kolkata,Banglore,CCU → MAA → BLR,10:20,4h,1 stop,No info,12,5,06,20
2,Jet Airways,Delhi,Cochin,DEL → BOM → COK,19:00 22 May,23h 45m,1 stop,In-flight meal not included,21,5,19,15
3,Multiple carriers,Delhi,Cochin,DEL → BOM → COK,21:00,13h,1 stop,No info,21,5,08,00
4,Air Asia,Banglore,Delhi,BLR → DEL,02:45 25 Jun,2h 50m,non-stop,No info,24,6,23,55


In [333]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2671 entries, 0 to 2670
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          2671 non-null   object
 1   Source           2671 non-null   object
 2   Destination      2671 non-null   object
 3   Route            2671 non-null   object
 4   Arrival_Time     2671 non-null   object
 5   Duration         2671 non-null   object
 6   Total_Stops      2671 non-null   object
 7   Additional_Info  2671 non-null   object
 8   day_journey      2671 non-null   int64 
 9   month_journey    2671 non-null   int64 
 10  hour             2671 non-null   object
 11  min              2671 non-null   object
dtypes: int64(2), object(10)
memory usage: 250.5+ KB


In [334]:
train.rename({'hour':'dep_hr'}, axis=1,inplace=True)
train.rename({'min':'dep_min'}, axis=1,inplace=True)
train['dep_hr']=train['dep_hr'].astype(int)
train['dep_min']=train['dep_min'].astype(int)
train.head(3)

,Airline,Source,Destination,Route,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,day_journey,month_journey,dep_hr,dep_min
0,IndiGo,Banglore,New Delhi,BLR → DEL,01:10 22 Mar,2h 50m,non-stop,No info,3897,24,3,22,20
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,13:15,7h 25m,2 stops,No info,7662,1,5,5,50
2,Jet Airways,Delhi,Cochin,DEL → LKO → BOM → COK,04:25 10 Jun,19h,2 stops,No info,13882,9,6,9,25


In [335]:
test.rename({'hour':'dep_hr'}, axis=1,inplace=True)
test.rename({'min':'dep_min'}, axis=1,inplace=True)
test['dep_hr']=test['dep_hr'].astype(int)
test['dep_min']=test['dep_min'].astype(int)
test.head(3)

,Airline,Source,Destination,Route,Arrival_Time,Duration,Total_Stops,Additional_Info,day_journey,month_journey,dep_hr,dep_min
0,Jet Airways,Delhi,Cochin,DEL → BOM → COK,04:25 07 Jun,10h 55m,1 stop,No info,6,6,17,30
1,IndiGo,Kolkata,Banglore,CCU → MAA → BLR,10:20,4h,1 stop,No info,12,5,6,20
2,Jet Airways,Delhi,Cochin,DEL → BOM → COK,19:00 22 May,23h 45m,1 stop,In-flight meal not included,21,5,19,15


In [336]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Source           10683 non-null  object
 2   Destination      10683 non-null  object
 3   Route            10682 non-null  object
 4   Arrival_Time     10683 non-null  object
 5   Duration         10683 non-null  object
 6   Total_Stops      10682 non-null  object
 7   Additional_Info  10683 non-null  object
 8   Price            10683 non-null  int64 
 9   day_journey      10683 non-null  int64 
 10  month_journey    10683 non-null  int64 
 11  dep_hr           10683 non-null  int32 
 12  dep_min          10683 non-null  int32 
dtypes: int32(2), int64(3), object(8)
memory usage: 1001.7+ KB


In [337]:
# 3. Arrival_time : we dont need the date part as we already took that from date of journey
# Arrival time is when the plane pulls up to the gate. # we can also take date of journey se arrival time if it had time also in it
train.Arrival_Time.unique()

array(['01:10 22 Mar', '13:15', '04:25 10 Jun', ..., '06:50 10 Mar',
       '00:05 19 Mar', '21:20 13 Mar'], dtype=object)

In [338]:
train['Arrival_hr']=pd.to_datetime(train['Arrival_Time']).dt.hour
train['Arrival_min']=pd.to_datetime(train['Arrival_Time']).dt.minute
train.head()

,Airline,Source,Destination,Route,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,day_journey,month_journey,dep_hr,dep_min,Arrival_hr,Arrival_min
0,IndiGo,Banglore,New Delhi,BLR → DEL,01:10 22 Mar,2h 50m,non-stop,No info,3897,24,3,22,20,1,10
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,13:15,7h 25m,2 stops,No info,7662,1,5,5,50,13,15
2,Jet Airways,Delhi,Cochin,DEL → LKO → BOM → COK,04:25 10 Jun,19h,2 stops,No info,13882,9,6,9,25,4,25
3,IndiGo,Kolkata,Banglore,CCU → NAG → BLR,23:30,5h 25m,1 stop,No info,6218,12,5,18,5,23,30
4,IndiGo,Banglore,New Delhi,BLR → NAG → DEL,21:35,4h 45m,1 stop,No info,13302,1,3,16,50,21,35


In [339]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2671 entries, 0 to 2670
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          2671 non-null   object
 1   Source           2671 non-null   object
 2   Destination      2671 non-null   object
 3   Route            2671 non-null   object
 4   Arrival_Time     2671 non-null   object
 5   Duration         2671 non-null   object
 6   Total_Stops      2671 non-null   object
 7   Additional_Info  2671 non-null   object
 8   day_journey      2671 non-null   int64 
 9   month_journey    2671 non-null   int64 
 10  dep_hr           2671 non-null   int32 
 11  dep_min          2671 non-null   int32 
dtypes: int32(2), int64(2), object(8)
memory usage: 229.7+ KB


In [340]:
test['Arrival_hr']=pd.to_datetime(test['Arrival_Time']).dt.hour
test['Arrival_min']=pd.to_datetime(test['Arrival_Time']).dt.minute
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2671 entries, 0 to 2670
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          2671 non-null   object
 1   Source           2671 non-null   object
 2   Destination      2671 non-null   object
 3   Route            2671 non-null   object
 4   Arrival_Time     2671 non-null   object
 5   Duration         2671 non-null   object
 6   Total_Stops      2671 non-null   object
 7   Additional_Info  2671 non-null   object
 8   day_journey      2671 non-null   int64 
 9   month_journey    2671 non-null   int64 
 10  dep_hr           2671 non-null   int32 
 11  dep_min          2671 non-null   int32 
 12  Arrival_hr       2671 non-null   int64 
 13  Arrival_min      2671 non-null   int64 
dtypes: int32(2), int64(4), object(8)
memory usage: 271.4+ KB


In [341]:
train.drop(['Arrival_Time'], axis=1, inplace=True)
test.drop(['Arrival_Time'], axis=1, inplace=True)

In [342]:
# 4. Duration :Time taken by plane to reach destination is called Duration
# It is the differnce betwwen Departure Time and Arrival time
durations =list(train['Duration'])
#this duration has some time in "hr min" format and some only in either hr or min so getting all values in the same range before extractiing hrs and mins sep from it 
for i in range(len(durations)): #range bcoz to repeat following fun for all records in the df
    if len(durations[i].split()) !=2: #getting min for only hours
        if 'h' in durations[i]:
            durations[i] =  durations[i] + ' 0m'
            print(durations[i])
        else:
            durations[i] =  '0h ' + durations[i]
            print(durations[i])
#now once we got this in same format now lets get hr and min in sep var
duration_hr = []
duration_min=[]
for i in range(len(durations)):
    duration_hr.append(int(durations[i].split()[0][:-1])) #take 2h from 2h Om and reomove h and take only 2
    duration_min.append(int(durations[i].split()[1][:-1]))#take 0m from 2h Om and reomove m and take only 0
    
train['duration_hr'] = duration_hr
train['duration_min'] = duration_min
train.head()

19h 0m
23h 0m
22h 0m
12h 0m
3h 0m
5h 0m
10h 0m
18h 0m
3h 0m
24h 0m
15h 0m
16h 0m
8h 0m
14h 0m
8h 0m
20h 0m
13h 0m
16h 0m
10h 0m
12h 0m
3h 0m
11h 0m
15h 0m
9h 0m
8h 0m
3h 0m
3h 0m
27h 0m
8h 0m
3h 0m
9h 0m
3h 0m
11h 0m
3h 0m
3h 0m
5h 0m
27h 0m
26h 0m
10h 0m
11h 0m
9h 0m
4h 0m
5h 0m
3h 0m
3h 0m
16h 0m
7h 0m
9h 0m
9h 0m
24h 0m
5h 0m
14h 0m
3h 0m
11h 0m
9h 0m
13h 0m
5h 0m
3h 0m
3h 0m
30h 0m
18h 0m
8h 0m
12h 0m
3h 0m
5h 0m
14h 0m
3h 0m
4h 0m
11h 0m
8h 0m
5h 0m
21h 0m
27h 0m
20h 0m
3h 0m
3h 0m
12h 0m
11h 0m
24h 0m
3h 0m
9h 0m
11h 0m
5h 0m
27h 0m
20h 0m
8h 0m
26h 0m
3h 0m
10h 0m
28h 0m
3h 0m
10h 0m
7h 0m
8h 0m
3h 0m
8h 0m
3h 0m
26h 0m
11h 0m
3h 0m
20h 0m
47h 0m
3h 0m
10h 0m
15h 0m
13h 0m
14h 0m
6h 0m
24h 0m
11h 0m
10h 0m
5h 0m
16h 0m
15h 0m
10h 0m
3h 0m
5h 0m
14h 0m
11h 0m
11h 0m
3h 0m
22h 0m
16h 0m
12h 0m
3h 0m
26h 0m
5h 0m
10h 0m
13h 0m
8h 0m
3h 0m
26h 0m
10h 0m
12h 0m
9h 0m
10h 0m
8h 0m
3h 0m
11h 0m
3h 0m
22h 0m
8h 0m
27h 0m
3h 0m
3h 0m
3h 0m
19h 0m
9h 0m
15h 0m
3h 0m
10h 0m
6h 0m
5h 0m
3h 

,Airline,Source,Destination,Route,Duration,Total_Stops,Additional_Info,Price,day_journey,month_journey,dep_hr,dep_min,Arrival_hr,Arrival_min,duration_hr,duration_min
0,IndiGo,Banglore,New Delhi,BLR → DEL,2h 50m,non-stop,No info,3897,24,3,22,20,1,10,2,50
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,7h 25m,2 stops,No info,7662,1,5,5,50,13,15,7,25
2,Jet Airways,Delhi,Cochin,DEL → LKO → BOM → COK,19h,2 stops,No info,13882,9,6,9,25,4,25,19,0
3,IndiGo,Kolkata,Banglore,CCU → NAG → BLR,5h 25m,1 stop,No info,6218,12,5,18,5,23,30,5,25
4,IndiGo,Banglore,New Delhi,BLR → NAG → DEL,4h 45m,1 stop,No info,13302,1,3,16,50,21,35,4,45


In [343]:
train.head()

,Airline,Source,Destination,Route,Duration,Total_Stops,Additional_Info,Price,day_journey,month_journey,dep_hr,dep_min,Arrival_hr,Arrival_min,duration_hr,duration_min
0,IndiGo,Banglore,New Delhi,BLR → DEL,2h 50m,non-stop,No info,3897,24,3,22,20,1,10,2,50
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,7h 25m,2 stops,No info,7662,1,5,5,50,13,15,7,25
2,Jet Airways,Delhi,Cochin,DEL → LKO → BOM → COK,19h,2 stops,No info,13882,9,6,9,25,4,25,19,0
3,IndiGo,Kolkata,Banglore,CCU → NAG → BLR,5h 25m,1 stop,No info,6218,12,5,18,5,23,30,5,25
4,IndiGo,Banglore,New Delhi,BLR → NAG → DEL,4h 45m,1 stop,No info,13302,1,3,16,50,21,35,4,45


In [344]:
#now lets do on the test data
# 4. Duration :Time taken by plane to reach destination is called Duration
# It is the differnce betwwen Departure Time and Arrival time
durations =list(test['Duration'])
#this duration has some time in "hr min" format and some only in either hr or min so getting all values in the same range before extractiing hrs and mins sep from it 
for i in range(len(durations)):
    if len(durations[i].split()) !=2: #getting min for only hours
        if 'h' in durations[i]:
            durations[i] =  durations[i] + ' 0m'
            print(durations[i])
        else:
            durations[i] =  '0h ' + durations[i]
            print(durations[i])
#now once we got this in same format now lets get hr and min in sep var
duration_hr = []
duration_min=[]
for i in range(len(durations)):
    duration_hr.append(int(durations[i].split()[0][:-1])) #take 2h from 2h Om and reomove h and take only 2
    duration_min.append(int(durations[i].split()[1][:-1]))#take 0m from 2h Om and reomove m and take only 0
    
test['duration_hr'] = duration_hr
test['duration_min'] = duration_min
test.head()

4h 0m
13h 0m
7h 0m
3h 0m
3h 0m
16h 0m
3h 0m
5h 0m
11h 0m
19h 0m
23h 0m
3h 0m
3h 0m
7h 0m
3h 0m
3h 0m
23h 0m
8h 0m
13h 0m
3h 0m
13h 0m
3h 0m
11h 0m
23h 0m
38h 0m
3h 0m
3h 0m
6h 0m
24h 0m
5h 0m
3h 0m
28h 0m
3h 0m
3h 0m
26h 0m
9h 0m
14h 0m
13h 0m
20h 0m
12h 0m
14h 0m
9h 0m
12h 0m
25h 0m
10h 0m
13h 0m
5h 0m
3h 0m
20h 0m
16h 0m
10h 0m
8h 0m
6h 0m
13h 0m
6h 0m
3h 0m
3h 0m
13h 0m
8h 0m
4h 0m
25h 0m
6h 0m
12h 0m
8h 0m
3h 0m
9h 0m
20h 0m
10h 0m
3h 0m
3h 0m
11h 0m
9h 0m
13h 0m
5h 0m
20h 0m
26h 0m
3h 0m
3h 0m
14h 0m
16h 0m
11h 0m
13h 0m
5h 0m
9h 0m
10h 0m
3h 0m
5h 0m
24h 0m
3h 0m
3h 0m
9h 0m
14h 0m
26h 0m
3h 0m
9h 0m
3h 0m
5h 0m
10h 0m
3h 0m
3h 0m
3h 0m
13h 0m
3h 0m
5h 0m
23h 0m
13h 0m
3h 0m
3h 0m
3h 0m
30h 0m
3h 0m
11h 0m
8h 0m
12h 0m
13h 0m
11h 0m
8h 0m
8h 0m
28h 0m
3h 0m
8h 0m
8h 0m
3h 0m
20h 0m
14h 0m
13h 0m
8h 0m
7h 0m
3h 0m
10h 0m
8h 0m
27h 0m
10h 0m
3h 0m
6h 0m
3h 0m
11h 0m
11h 0m
11h 0m
24h 0m
8h 0m
12h 0m
28h 0m
9h 0m
15h 0m
28h 0m
3h 0m
3h 0m
28h 0m
3h 0m
14h 0m
9h 0m
18h 0m
3h 0m
27h 0

,Airline,Source,Destination,Route,Duration,Total_Stops,Additional_Info,day_journey,month_journey,dep_hr,dep_min,Arrival_hr,Arrival_min,duration_hr,duration_min
0,Jet Airways,Delhi,Cochin,DEL → BOM → COK,10h 55m,1 stop,No info,6,6,17,30,4,25,10,55
1,IndiGo,Kolkata,Banglore,CCU → MAA → BLR,4h,1 stop,No info,12,5,6,20,10,20,4,0
2,Jet Airways,Delhi,Cochin,DEL → BOM → COK,23h 45m,1 stop,In-flight meal not included,21,5,19,15,19,0,23,45
3,Multiple carriers,Delhi,Cochin,DEL → BOM → COK,13h,1 stop,No info,21,5,8,0,21,0,13,0
4,Air Asia,Banglore,Delhi,BLR → DEL,2h 50m,non-stop,No info,24,6,23,55,2,45,2,50


In [345]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2671 entries, 0 to 2670
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          2671 non-null   object
 1   Source           2671 non-null   object
 2   Destination      2671 non-null   object
 3   Route            2671 non-null   object
 4   Duration         2671 non-null   object
 5   Total_Stops      2671 non-null   object
 6   Additional_Info  2671 non-null   object
 7   day_journey      2671 non-null   int64 
 8   month_journey    2671 non-null   int64 
 9   dep_hr           2671 non-null   int32 
 10  dep_min          2671 non-null   int32 
 11  Arrival_hr       2671 non-null   int64 
 12  Arrival_min      2671 non-null   int64 
 13  duration_hr      2671 non-null   int64 
 14  duration_min     2671 non-null   int64 
dtypes: int32(2), int64(6), object(7)
memory usage: 292.3+ KB


In [346]:
train.drop('Duration', axis=1, inplace = True)
test.drop('Duration', axis=1, inplace = True)

In [347]:
train.head(3)

,Airline,Source,Destination,Route,Total_Stops,Additional_Info,Price,day_journey,month_journey,dep_hr,dep_min,Arrival_hr,Arrival_min,duration_hr,duration_min
0,IndiGo,Banglore,New Delhi,BLR → DEL,non-stop,No info,3897,24,3,22,20,1,10,2,50
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,2 stops,No info,7662,1,5,5,50,13,15,7,25
2,Jet Airways,Delhi,Cochin,DEL → LKO → BOM → COK,2 stops,No info,13882,9,6,9,25,4,25,19,0


In [348]:
# Route variable : its shows the destination route for that particualr flight but same info is conveyed in more easier way in the total_stops col
#so taking total_stops col instead which is more easier to interpret
train.Total_Stops.unique()

array(['non-stop', '2 stops', '1 stop', '3 stops', nan, '4 stops'],
      dtype=object)

In [349]:
#now as we see if stops increase price increase so, it be considered as an ordinal columns with more stops as highest order
train.groupby('Total_Stops')["Price"].mean() 
# Total_Stops
# 1 stop      10594.123556
# 2 stops     12715.807895
# 3 stops     13112.000000
# 4 stops     17686.000000
# non-stop     5024.900315

Total_Stops
1 stop      10594.123556
2 stops     12715.807895
3 stops     13112.000000
4 stops     17686.000000
non-stop     5024.900315
Name: Price, dtype: float64

In [350]:
# As this is case of Ordinal Categorical type we perform LabelEncoder
# Here Values are assigned with corresponding keys
train['Total_Stops'] = train['Total_Stops'].map({'non-stop':0, '2 stops':2, '1 stop':1, '3 stops':3, 'nan':1, '4 stops':4})
train.head()

,Airline,Source,Destination,Route,Total_Stops,Additional_Info,Price,day_journey,month_journey,dep_hr,dep_min,Arrival_hr,Arrival_min,duration_hr,duration_min
0,IndiGo,Banglore,New Delhi,BLR → DEL,0.0,No info,3897,24,3,22,20,1,10,2,50
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,2.0,No info,7662,1,5,5,50,13,15,7,25
2,Jet Airways,Delhi,Cochin,DEL → LKO → BOM → COK,2.0,No info,13882,9,6,9,25,4,25,19,0
3,IndiGo,Kolkata,Banglore,CCU → NAG → BLR,1.0,No info,6218,12,5,18,5,23,30,5,25
4,IndiGo,Banglore,New Delhi,BLR → NAG → DEL,1.0,No info,13302,1,3,16,50,21,35,4,45


In [351]:
test['Total_Stops'] = test['Total_Stops'].map({'non-stop':0, '2 stops':2, '1 stop':1, '3 stops':3, 'nan':1, '4 stops':4})
test.head(3)

,Airline,Source,Destination,Route,Total_Stops,Additional_Info,day_journey,month_journey,dep_hr,dep_min,Arrival_hr,Arrival_min,duration_hr,duration_min
0,Jet Airways,Delhi,Cochin,DEL → BOM → COK,1,No info,6,6,17,30,4,25,10,55
1,IndiGo,Kolkata,Banglore,CCU → MAA → BLR,1,No info,12,5,6,20,10,20,4,0
2,Jet Airways,Delhi,Cochin,DEL → BOM → COK,1,In-flight meal not included,21,5,19,15,19,0,23,45


In [352]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2671 entries, 0 to 2670
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          2671 non-null   object
 1   Source           2671 non-null   object
 2   Destination      2671 non-null   object
 3   Route            2671 non-null   object
 4   Total_Stops      2671 non-null   int64 
 5   Additional_Info  2671 non-null   object
 6   day_journey      2671 non-null   int64 
 7   month_journey    2671 non-null   int64 
 8   dep_hr           2671 non-null   int32 
 9   dep_min          2671 non-null   int32 
 10  Arrival_hr       2671 non-null   int64 
 11  Arrival_min      2671 non-null   int64 
 12  duration_hr      2671 non-null   int64 
 13  duration_min     2671 non-null   int64 
dtypes: int32(2), int64(7), object(5)
memory usage: 271.4+ KB


In [353]:
train.drop('Route', axis=1, inplace = True)
test.drop('Route', axis=1, inplace = True)

In [354]:
train.isnull().sum()

Airline            0
Source             0
Destination        0
Total_Stops        1
Additional_Info    0
Price              0
day_journey        0
month_journey      0
dep_hr             0
dep_min            0
Arrival_hr         0
Arrival_min        0
duration_hr        0
duration_min       0
dtype: int64

In [355]:
train['Total_Stops']=np.where(train['Total_Stops'].isnull(),1,0)
train.isnull().sum()

Airline            0
Source             0
Destination        0
Total_Stops        0
Additional_Info    0
Price              0
day_journey        0
month_journey      0
dep_hr             0
dep_min            0
Arrival_hr         0
Arrival_min        0
duration_hr        0
duration_min       0
dtype: int64

In [356]:
test.isnull().sum()

Airline            0
Source             0
Destination        0
Total_Stops        0
Additional_Info    0
day_journey        0
month_journey      0
dep_hr             0
dep_min            0
Arrival_hr         0
Arrival_min        0
duration_hr        0
duration_min       0
dtype: int64

In [357]:
train.shape, test.shape

((10683, 14), (2671, 13))

In [290]:
train.head(3)

,Airline,Source,Destination,Total_Stops,Additional_Info,Price,day_journey,month_journey,dep_hr,dep_min,Arrival_hr,Arrival_min,duration_hr,duration_min
0,IndiGo,Banglore,New Delhi,0,No info,3897,24,3,22,20,1,10,2,50
1,Air India,Kolkata,Banglore,0,No info,7662,1,5,5,50,13,15,7,25
2,Jet Airways,Delhi,Cochin,0,No info,13882,9,6,9,25,4,25,19,0


In [291]:
# Additional Info
train.Additional_Info.unique()

array(['No info', 'In-flight meal not included',
       'No check-in baggage included', '1 Short layover', 'No Info',
       '1 Long layover', 'Change airports', 'Business class',
       'Red-eye flight', '2 Long layover'], dtype=object)

In [292]:
train.Additional_Info.value_counts()

No info                         8345
In-flight meal not included     1982
No check-in baggage included     320
1 Long layover                    19
Change airports                    7
Business class                     4
No Info                            3
1 Short layover                    1
Red-eye flight                     1
2 Long layover                     1
Name: Additional_Info, dtype: int64

In [293]:
train.groupby('Source')['Price'].mean()

Source
Banglore     8017.464269
Chennai      4789.892388
Delhi       10539.439057
Kolkata      9158.389411
Mumbai       5059.708752
Name: Price, dtype: float64

In [294]:
train.groupby('Destination')['Price'].mean()

Destination
Banglore      9158.389411
Cochin       10539.439057
Delhi         5143.918577
Hyderabad     5059.708752
Kolkata       4789.892388
New Delhi    11917.716738
Name: Price, dtype: float64

In [359]:
#performing label enmcoding for source and destination
X_train = train.drop('Price',axis=1)
y_train=train['Price']
X_test=test
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X_train['Source'] = le.fit_transform(X_train['Source'])
X_train['Destination'] = le.fit_transform(X_train['Destination'])
X_train['Additional_Info'] = le.fit_transform(X_train['Additional_Info'])
X_train['Airline'] = le.fit_transform(X_train['Airline'])

In [360]:
X_test['Source'] = le.fit_transform(X_test['Source'])
X_test['Destination'] = le.fit_transform(X_test['Destination'])
X_test['Additional_Info'] = le.fit_transform(X_test['Additional_Info'])
X_test['Airline'] = le.fit_transform(X_test['Airline'])

In [361]:
X_train.head(3)

,Airline,Source,Destination,Total_Stops,Additional_Info,day_journey,month_journey,dep_hr,dep_min,Arrival_hr,Arrival_min,duration_hr,duration_min
0,3,0,5,0,8,24,3,22,20,1,10,2,50
1,1,3,0,0,8,1,5,5,50,13,15,7,25
2,4,2,1,0,8,9,6,9,25,4,25,19,0


In [362]:
X_test.head(3)

,Airline,Source,Destination,Total_Stops,Additional_Info,day_journey,month_journey,dep_hr,dep_min,Arrival_hr,Arrival_min,duration_hr,duration_min
0,4,2,1,1,5,6,6,17,30,4,25,10,55
1,3,3,0,1,5,12,5,6,20,10,20,4,0
2,4,2,1,1,3,21,5,19,15,19,0,23,45


In [363]:
X_train.shape, X_test.shape

((10683, 13), (2671, 13))

In [364]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   Airline          10683 non-null  int32
 1   Source           10683 non-null  int32
 2   Destination      10683 non-null  int32
 3   Total_Stops      10683 non-null  int32
 4   Additional_Info  10683 non-null  int32
 5   day_journey      10683 non-null  int64
 6   month_journey    10683 non-null  int64
 7   dep_hr           10683 non-null  int32
 8   dep_min          10683 non-null  int32
 9   Arrival_hr       10683 non-null  int64
 10  Arrival_min      10683 non-null  int64
 11  duration_hr      10683 non-null  int64
 12  duration_min     10683 non-null  int64
dtypes: int32(7), int64(6)
memory usage: 793.0 KB


In [365]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2671 entries, 0 to 2670
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   Airline          2671 non-null   int32
 1   Source           2671 non-null   int32
 2   Destination      2671 non-null   int32
 3   Total_Stops      2671 non-null   int64
 4   Additional_Info  2671 non-null   int32
 5   day_journey      2671 non-null   int64
 6   month_journey    2671 non-null   int64
 7   dep_hr           2671 non-null   int32
 8   dep_min          2671 non-null   int32
 9   Arrival_hr       2671 non-null   int64
 10  Arrival_min      2671 non-null   int64
 11  duration_hr      2671 non-null   int64
 12  duration_min     2671 non-null   int64
dtypes: int32(6), int64(7)
memory usage: 208.8 KB


In [310]:
for i in X_test.columns:
    if X_test[i].dtype=='uint8':
           X_test[i] =X_test[i].astype(int)
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2671 entries, 0 to 2670
Data columns (total 26 columns):
 #   Column                             Non-Null Count  Dtype
---  ------                             --------------  -----
 0   Source                             2671 non-null   int32
 1   Destination                        2671 non-null   int32
 2   Total_Stops                        2671 non-null   int64
 3   day_journey                        2671 non-null   int64
 4   month_journey                      2671 non-null   int64
 5   dep_hr                             2671 non-null   int32
 6   dep_min                            2671 non-null   int32
 7   Arrival_hr                         2671 non-null   int64
 8   Arrival_min                        2671 non-null   int64
 9   duration_hr                        2671 non-null   int64
 10  duration_min                       2671 non-null   int64
 11  Air India                          2671 non-null   int32
 12  GoAir               

In [366]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)

In [ ]:
#now its ready for future use